In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Install tmdbsimple (only need to run once)
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 78.4/78.4 KB 2.2 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Program Files\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
# Load API Credentials
with open('C:/Users/joshu/.secret/yelp_api.json') as f: 
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
# set our API call parameters and filename before the first call
LOCATION = 'Baltimore, MD,21202'
TERM = 'Crab Cakes'

In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_Crab_cakes.json"
JSON_FILE

'Data/results_in_progress_Crab_cakes.json'

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Crab_cakes.json already exists.


## Determine how many results are already in the file

In [9]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [10]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
## How many results total?
total_results = results['total']
total_results

439

In [12]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [13]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

22

In [14]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [16]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/22 [00:00<?, ?it/s]

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/22 [00:00<?, ?it/s]

In [18]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,D9A33FM394q99o4QtK5YwA,faidleys-seafood-baltimore-3,Faidley's Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/8j3ynZ...,False,https://www.yelp.com/biz/faidleys-seafood-balt...,1232,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 39.291696, 'longitude': -76.62224}",[delivery],$$,"{'address1': '203 N Paca St', 'address2': '', ...",+14107274898,(410) 727-4898,1349.560720
1,ieS_5zqxDHcWMCm8BKUYbg,thames-street-oyster-house-baltimore,Thames Street Oyster House,https://s3-media1.fl.yelpcdn.com/bphoto/9hGjo5...,False,https://www.yelp.com/biz/thames-street-oyster-...,2848,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 39.28214, 'longitude': -76.59162}",[delivery],$$$,"{'address1': '1728 Thames St', 'address2': '',...",+14434497726,(443) 449-7726,2090.712792
2,u65W69AhbjUlvJJBkEhGNQ,miss-shirleys-cafe-baltimore-9,Miss Shirley's Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Guq6JM...,False,https://www.yelp.com/biz/miss-shirleys-cafe-ba...,3071,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 39.2870995, 'longitude': -76.6053...",[delivery],$$,"{'address1': '750 E Pratt St', 'address2': '',...",+14105285373,(410) 528-5373,1028.736468
3,RoTOf5IY32JAdEKoKjycKg,werner-s-diner-and-pub-baltimore,Werner’s Diner and Pub,https://s3-media4.fl.yelpcdn.com/bphoto/fwS2lm...,False,https://www.yelp.com/biz/werner-s-diner-and-pu...,43,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 39.28885259491462, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '231 E Redwood St', 'address2': '...",+14102307061,(410) 230-7061,876.944503
4,6am8TZAFnvND52MOz-Yctg,mamas-on-the-half-shell-baltimore,Mama's On The Half Shell,https://s3-media2.fl.yelpcdn.com/bphoto/HWY8OF...,False,https://www.yelp.com/biz/mamas-on-the-half-she...,1324,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 39.27986, 'longitude': -76.5752399}","[delivery, pickup]",$$,"{'address1': '2901 Odonnell St', 'address2': '...",+14102763160,(410) 276-3160,3328.825798


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
426,UvLC7ZEL7mR57ofbusy1EA,eastern-pizza-baltimore,Eastern Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/9PXlH9...,False,https://www.yelp.com/biz/eastern-pizza-baltimo...,34,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 39.28725, 'longitude': -76.55671}","[delivery, pickup]",$$,"{'address1': '4616 Eastern Ave', 'address2': '...",+14105227212,(410) 522-7212,4496.394084
427,EuoMgEcBmsZfc-oPY07tag,charles-village-pizza-and-sub-baltimore,Charles Village Pizza & Sub,https://s3-media1.fl.yelpcdn.com/bphoto/cuuB5f...,False,https://www.yelp.com/biz/charles-village-pizza...,67,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",3.5,"{'latitude': 39.31728, 'longitude': -76.61501}","[delivery, pickup]",$$,"{'address1': '2435 St Paul St', 'address2': ''...",+14103661600,(410) 366-1600,2432.309163
428,dWI0jCoOqjge9VQTIad-Ww,york-pizza-subs-and-wings-baltimore,York Pizza Subs and Wings,https://s3-media4.fl.yelpcdn.com/bphoto/Ay3jyi...,False,https://www.yelp.com/biz/york-pizza-subs-and-w...,60,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"{'latitude': 39.3298530578613, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '3401 Greenmount Ave', 'address2'...",+14102356290,(410) 235-6290,3743.767034
429,rtlLrtUn35c9Y2eIcxHT8g,kings-pizza-and-subs-baltimore-2,King's Pizza & Subs,https://s3-media2.fl.yelpcdn.com/bphoto/9FEdeU...,False,https://www.yelp.com/biz/kings-pizza-and-subs-...,75,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.0,"{'latitude': 39.33086, 'longitude': -76.63161}","[delivery, pickup]",$$,"{'address1': '907 W 36th St', 'address2': None...",+14108893663,(410) 889-3663,4382.210159
430,EWkA_yQ1VPa7cHbkiBbawA,phillys-best-baltimore,Philly's Best,https://s3-media4.fl.yelpcdn.com/bphoto/CXQdUz...,False,https://www.yelp.com/biz/phillys-best-baltimor...,48,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 39.33084, 'longitude': -76.63393}","[delivery, pickup]",$,"{'address1': '1101 W 36th St', 'address2': '',...",+14102431301,(410) 243-1301,4467.724059


In [19]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

0

In [20]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_crab_cakes.csv.gz', compression='gzip',index=False)